In [9]:
import psycopg2
from collections import defaultdict
import json

In [10]:
conn = psycopg2.connect(
    dbname="beta_base_db",
    user="psqlsynthadmin",
    password="PSQL$ynth#2024",
    host="base-report-synth-postgredb.cb6ae20mupdy.ap-south-1.rds.amazonaws.com",
    port="5432"
)
cur = conn.cursor()
print("✅ Connected to PostgreSQL")

✅ Connected to PostgreSQL


In [34]:
# Replace with your actual classes table
cur.execute("SELECT id, name FROM public.generic_master;")
classes_raw = cur.fetchall()

# Store class names
classes = [row[1] for row in classes_raw]
print("🎓 Classes:", classes)

🎓 Classes: ['Course', 'Program', 'Admission Year', 'Academic Term', 'Subject', 'NAAC Accreditation', 'Teaching Learning and Evaluation', 'NBA Accreditation', 'Outcome-Based Education (OBE)', 'Exam Type', 'Exam Name', 'Topics', 'Sections', 'Chapters', 'Sub Category', 'Curriculum Year', 'Courses Type', 'ELMS Course', 'Academic Year', 'Building', 'Floor', 'Class Room', 'Batch', 'School Subject', 'Faculty', 'Class', 'Division']


In [35]:
# Get subject master ID
cur.execute("SELECT id FROM public.generic_master WHERE name ILIKE 'subject';")
subject_master_id = cur.fetchone()[0]

# Now get subject data
cur.execute(f"SELECT id, name FROM public.generic_master_data WHERE master_id = {subject_master_id};")
subject_data = cur.fetchall()

subject_id_name_map = {row[0]: row[1] for row in subject_data}
subjects = list(subject_id_name_map.values())

print("📘 Subjects:", subjects)
print(f"subject_master_id: {subject_master_id}")
print(f"subject master data :{subject_data}")


📘 Subjects: ['Mathematics-I', 'ENGINEERING PHYSICS - I', 'Engineering Chemistry', 'Data Communication & Computer Networks', 'Physics Chemistry Lab - I', 'Math-1', 'Advanced Programming Concepts', 'data structures and algorithms', 'Physics Laboratory', 'Object Oriented Programming', 'Civil-Math1', 'Basics of Computer Network', 'Maths', 'Control Systems', 'Environment Science ', 'Database Management System', 'Engineering Graphics and Design ', 'Material Science', 'History', 'Chemical Process Principles', 'Angular', 'Industrial Management and Safety', 'Angular', 'Ionic', 'Biology', 'Chem', 'Environment', 'Social Science ', 'Physics I', 'Updated Mathematics', 'Angular', 'Lavaras', 'Grammer', 'Grammer', 'Hindi ', 'French', 'Jermany', 'Vue', 'ChatBoot', 'Electronic Circuits-204181', '22426-Microcontroller and Applications', 'Pharmaceutical analysis-I', 'Pharmaceutics I', 'Pharmaceutical Inorganic Chemistry', 'Pharmaceutical Organic Chemistry I', 'English', 'Marathi', 'Math', 'History', 'Engl

In [4]:
# Step 1: Get subject ID → subject name
cur.execute("SELECT id, name FROM public.generic_master_data;")
subject_id_name_map = dict(cur.fetchall())

# Step 2: Get faculty info from subject_faculty
cur.execute("""
    SELECT 
        faculty_id, 
        first_name || ' ' || last_name AS faculty_name, 
        subject_id
    FROM public.subject_faculty;
""")
rows = cur.fetchall()

# Step 3: Build the teacher → subjects mapping
from collections import defaultdict

teacher_to_subjects = defaultdict(list)

for faculty_id, faculty_name, subject_id in rows:
    subject_name = subject_id_name_map.get(subject_id)
    if subject_name:
        teacher_to_subjects[faculty_name].append(subject_name)

# Output the mapping
print("📚 Teacher → Subjects mapping:")
for teacher, subjects in teacher_to_subjects.items():
    print(f"{teacher}: {subjects}")


📚 Teacher → Subjects mapping:
Pratik More: ['Chem', 'Physics I-Internal']
Sagar Patil: ['Updated Mathematics']
Rohan Suradkar: ['Physics I Univercity Exam-8967']
Aafreen Kotadiya: ['Social Science ', 'Marathi', 'Math']
Aditi Bulbule: ['ChatBoot', 'Hindi ']
Abhijeet Wagh: ['Jermany']
Abhidnya Gaikwad: ['Vue']
Bhavana Tambe: ['Grammer']
Atul Baviskar: ['English']
Arvind Kumbhar: ['English', 'Math']
Harshada Phate: ['Math']
Jayesh Dhabalia: ['EVS', 'Hindi']
Keyur Karad: ['Drawing']
Mrinmayee Sonawane: ['English']
Nirmala Shinde: ['Math']
Pradeep Jadhav: ['EVS']
Priya Wakchaure: ['Hindi']


In [51]:
# cur.execute("""
#     SELECT f.id AS faculty_id, f.name AS faculty_name, gmd.name AS subject_name
#     FROM public.subject_faculty sf
#     JOIN public.generic_master_data gmd ON sf.subject_id = gmd.id
#     JOIN public.faculty_master f ON sf.faculty_id = f.id;
# """)
# rows = cur.fetchall()

# # Build teacher → subjects map
# from collections import defaultdict

# teacher_to_subjects = defaultdict(list)
# for faculty_id, faculty_name, subject_name in rows:
#     teacher_to_subjects[faculty_name].append(subject_name)

# # Print
# print("📚 Teacher → Subjects mapping:")
# for teacher, subs in teacher_to_subjects.items():
#     print(f"{teacher}: {subs}")


In [53]:
# conn.rollback()

# Setting up the classroom for a particular classroom 

In [5]:
# Step 1: Get Class-Division pairs
cur.execute("""
    SELECT d.id AS division_id, d.name AS division_name,
           c.id AS class_id, c.name AS class_name
    FROM public.generic_master_data d
    JOIN public.generic_master_data c
      ON d.parent_id = c.id
    WHERE d.master_id = 24 AND c.master_id = 23;
""")
class_division_rows = cur.fetchall()

# Create mapping: e.g., {'6A': {'class_id': ..., 'division_id': ...}, ...}
class_division_map = {}
for division_id, division_name, class_id, class_name in class_division_rows:
    key = f"{class_name}{division_name}"
    class_division_map[key] = {
        'class_id': class_id,
        'division_id': division_id
    }

# Step 2: Get Classrooms
cur.execute("""
    SELECT id, name
    FROM public.generic_master_data
    WHERE master_id = 22;
""")
classroom_rows = cur.fetchall()
classroom_dict = {row[0]: row[1] for row in classroom_rows}

# Preview results
print("📘 Class-Division Map:")
for key, val in class_division_map.items():
    print(f"{key}: {val}")

print("\n🏫 Available Classrooms:")
for cid, cname in classroom_dict.items():
    print(f"{cid}: {cname}")


📘 Class-Division Map:
8 th Div - A: {'class_id': 130, 'division_id': 131}

🏫 Available Classrooms:
129: 8A - The Innovation Room
138: 101 A
139: 102B


In [7]:
cur.execute("""
    SELECT d.id AS division_id, d.name AS division_name,
           c.id AS class_id, c.name AS class_name
    FROM public.generic_master_data d
    JOIN public.generic_master_data c
      ON d.parent_id = c.id
    WHERE d.master_id = 24 AND c.master_id = 23;
""")
class_division_rows = cur.fetchall()



(131, 'Div - A', 130, '8 th ')


In [11]:
query = """
    SELECT c.name AS class_name, d.name AS division_name
    FROM generic_master_data d
    JOIN generic_master_data c ON d.parent_id = c.id
    WHERE c.master_id = 23 AND d.master_id = 24 AND c.name = %s
"""
cur.execute(query, (class_name,))


In [13]:
cur.execute("""
        SELECT d.id AS division_id, d.name AS division_name,
               c.id AS class_id, c.name AS class_name
        FROM public.generic_master_data d
        JOIN public.generic_master_data c ON d.parent_id = c.id
        WHERE d.master_id = 24 AND c.master_id = 23;
    """)
class_division_rows = cur.fetchall()
class_division_map = {}
for division_id, division_name, class_id, class_name in class_division_rows:
    key = f"{class_name}  Div - {division_name}"
    class_division_map[key] = {
            'class_id': class_id,
            'division_id': division_id
        }

In [17]:
class_division_map

{'8 th   Div - Div - A': {'class_id': 130,
  'division_id': 131,
  'classroom_id': None,
  'classroom_name': 'Unknown'}}

In [15]:
for key, val in class_division_map.items():
        division_id = val["division_id"]
        classroom_id = class_division_map.get(division_id)
        val["classroom_id"] = classroom_id
        val["classroom_name"] = classroom_dict.get(classroom_id, "Unknown")

In [16]:
class_division_map.items()

dict_items([('8 th   Div - Div - A', {'class_id': 130, 'division_id': 131, 'classroom_id': None, 'classroom_name': 'Unknown'})])

In [ ]:
def get_divisions_for_class(class_name: str):
    conn = psycopg2.connect(
        dbname="beta_base_db",
        user="psqlsynthadmin",
        password="PSQL$ynth#2024",
        host="base-report-synth-postgredb.cb6ae20mupdy.ap-south-1.rds.amazonaws.com",
        port="5432"
    )
    cur = conn.cursor()

    query = """
    SELECT c.name AS class_name, d.name AS division_name
    FROM generic_master_data d
    JOIN generic_master_data c ON d.parent_id = c.id
    WHERE c.master_id = 5 AND d.master_id = 24 AND c.name = %s
    """
    cur.execute(query, (class_name,))
    rows = cur.fetchall()
    cur.close()
    conn.close()
    # print(rows)

    return [f"{row[0]}  Div - {row[1]}" for row in rows]

In [38]:
get_divisions_for_class("3 rd std")

[('3 rd std', 'A  div'), ('3 rd std', 'B  div'), ('3 rd std', 'C  div')]


['3 rd std  Div - A  div', '3 rd std  Div - B  div', '3 rd std  Div - C  div']

In [35]:
import psycopg2

def get_classes():
    conn = psycopg2.connect(
        dbname="beta_base_db",
        user="psqlsynthadmin",
        password="PSQL$ynth#2024",
        host="base-report-synth-postgredb.cb6ae20mupdy.ap-south-1.rds.amazonaws.com",
        port="5432"
    )
    cur = conn.cursor()

    query = """
    SELECT id, name 
    FROM public.generic_master_data 
    WHERE master_id = 5;
    """
    cur.execute(query)
    rows = cur.fetchall()

    cur.close()
    conn.close()

    return [{"id": row[0], "name": row[1]} for row in rows]

get_classes()

[{'id': 75, 'name': 'MCA 1st semester'},
 {'id': 76, 'name': 'BE 1st semester'},
 {'id': 66, 'name': 'MCA-1 sem'},
 {'id': 6, 'name': 'BE ME 1st Semester'},
 {'id': 100, 'name': 'BE-ME-2nd-1 sem'},
 {'id': 113, 'name': 'BE-Computer Science- 1 sem'},
 {'id': 117, 'name': 'BE-IT-3 Sem'},
 {'id': 124, 'name': 'Civil- 1 sem'},
 {'id': 145, 'name': 'NA'},
 {'id': 146, 'name': '8th Std'},
 {'id': 147, 'name': '9th Std'},
 {'id': 155, 'name': 'BE-Electronics and Telecommunication-3 Sem'},
 {'id': 162, 'name': '10 th '},
 {'id': 163, 'name': '2 st'},
 {'id': 169, 'name': 'Electrical 1st semester'},
 {'id': 184, 'name': 'Electrical-3rd semester'},
 {'id': 199, 'name': 'BE-Chemical Engineering-3rd sem'},
 {'id': 206, 'name': 'Diploma-chemical-1sem'},
 {'id': 232, 'name': 'Msc '},
 {'id': 256, 'name': 'Sem-I'},
 {'id': 276, 'name': '1 st '},
 {'id': 292, 'name': '9 th Std'},
 {'id': 294, 'name': '3 rd std'},
 {'id': 295, 'name': '4 th std'},
 {'id': 296, 'name': '5 th std'}]